## Complete tree printing

In [14]:
import re

def parse_log_file(file_path):
    with open(file_path, 'r') as f:
        logs = []
        for line in f:
            # Match for function entry logs with '>' or '-->'
            match_push = re.match(r'^(--+>|\s*>)\s*call function (.+?) in (.+?):(\d+)', line)
            if match_push:
                function_name = match_push.group(2)
                file_path = match_push.group(3)
                line_number = int(match_push.group(4))
                logs.append((function_name, file_path, line_number, "push"))
                continue

            # Match for function exit logs with '<' or '<--'
            match_pop = re.match(r'^(<--+|\s*<)\s*exit function (.+?) in (.+?):(\d+)', line)
            if match_pop:
                function_name = match_pop.group(2)
                file_path = match_pop.group(3)
                line_number = int(match_pop.group(4))
                logs.append((function_name, file_path, line_number, "pop"))
                continue

    return logs

class CallStackNode:
    def __init__(self, function_name, file_path, line_number):
        self.function_name = function_name
        self.file_path = file_path
        self.line_number = line_number
        self.children = []
        self.parent = None

    def add_child(self, child_node):
        child_node.parent = self
        self.children.append(child_node)

    def __repr__(self, level=0, is_last_child=True, parent_last_childs=[]):
        indent = ''
        for parent_last in parent_last_childs:
            indent += '    ' if parent_last else '│   '

        branch = '└── ' if is_last_child else '├── '
        repr_str = f"{indent}{branch}{self.function_name}, {self.file_path}:{self.line_number}\n" # complete
        # repr_str = f"{indent}{branch}{self.function_name}\n" # simple without file path and line number

        for i, child in enumerate(self.children):
            repr_str += child.__repr__(level + 1, i == len(self.children) - 1, parent_last_childs + [is_last_child])
        return repr_str

class CallStackTree:
    def __init__(self):
        self.root = CallStackNode("root", "", 0)
        self.current_node = self.root

    def push(self, function_name, file_path, line_number):
        new_node = CallStackNode(function_name, file_path, line_number)
        self.current_node.add_child(new_node)
        self.current_node = new_node

    def pop(self):
        if self.current_node != self.root:
            self.current_node = self.current_node.parent

    def __repr__(self):
        return self.root.__repr__()

# Parse log file and construct call stack tree
log_file_path = "/root/vescale_prj/veScale/test/parallel/pipeline/instruction/logs-test_schedule-host_f78e8e970a17-pid_199624-py/tracing-test_schedule-20240829_071457.log"
logs = parse_log_file(log_file_path)

call_stack_tree = CallStackTree()

for log in logs:
    function_name, file_path, line_number, operation = log
    if operation == "push":
        call_stack_tree.push(function_name, file_path, line_number)
    elif operation == "pop":
        call_stack_tree.pop()

print(call_stack_tree)

└── root, :0
    ├── init_device_mesh, /root/vescale_prj/veScale/vescale/devicemesh_api/api.py:48
    │   └── init_device_mesh, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:594
    │       └── __init__, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:224
    │           ├── update_vescale_debug_mode_from_env, /root/vescale_prj/veScale/vescale/debug/debug_log.py:88
    │           ├── _get_or_create_default_group, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:313
    │           │   └── _get_device_handle, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:153
    │           ├── _get_device_handle, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:153
    │           ├── _get_current_device, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:281
    │           ├── _validate_mesh, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:339
    │           └── _init_process_groups, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.

Install 

```bash
sudo apt-get update
sudo apt-get install python3-tk
pip install nest_asyncio ipywidgets
```

### load at once with good UI

In [ ]:
import tkinter as tk
from tkinter import ttk, scrolledtext
import re
import nest_asyncio
import os
from IPython.display import display

nest_asyncio.apply()

# Use the provided code to parse the log file and create a call stack tree

def parse_log_file(file_path):
    with open(file_path, 'r') as f:
        logs = []
        for line in f:
            match_push = re.match(r'^(--+>|\s*>)\s*call function (.+?) in (.+?):(\d+)', line)
            if match_push:
                function_name = match_push.group(2)
                file_path = match_push.group(3)
                line_number = int(match_push.group(4))
                logs.append((function_name, file_path, line_number, "push"))
                continue

            match_pop = re.match(r'^(<--+|\s*<)\s*exit function (.+?) in (.+?):(\d+)', line)
            if match_pop:
                function_name = match_pop.group(2)
                file_path = match_pop.group(3)
                line_number = int(match_pop.group(4))
                logs.append((function_name, file_path, line_number, "pop"))
                continue

    return logs

class CallStackNode:
    def __init__(self, function_name, file_path, line_number):
        self.function_name = function_name
        self.file_path = file_path
        self.line_number = line_number
        self.children = []
        self.parent = None

    def add_child(self, child_node):
        child_node.parent = self
        self.children.append(child_node)

class CallStackTree:
    def __init__(self):
        self.root = CallStackNode("root", "", 0)
        self.current_node = self.root

    def push(self, function_name, file_path, line_number):
        new_node = CallStackNode(function_name, file_path, line_number)
        self.current_node.add_child(new_node)
        self.current_node = new_node

    def pop(self):
        if self.current_node != self.root:
            self.current_node = self.current_node.parent

# GUI Application

class CodeReviewApp:
    def __init__(self, root, call_stack_tree):
        self.root = root
        self.root.title("Code Review App")
        self.call_stack_tree = call_stack_tree

        # Treeview for displaying call stack
        self.tree = ttk.Treeview(root)
        self.tree.heading("#0", text="Call Stack", anchor='w')
        self.tree.pack(side=tk.LEFT, fill=tk.Y)

        # ScrolledText for displaying code snippets
        self.code_display = scrolledtext.ScrolledText(root, wrap=tk.WORD)
        self.code_display.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

        # Populate treeview with call stack nodes
        self.populate_tree(self.tree, "", self.call_stack_tree.root)

        # Bind treeview selection event
        self.tree.bind("<<TreeviewSelect>>", self.on_tree_select)

    def populate_tree(self, tree, parent, node):
        for child in node.children:
            tree_node = tree.insert(parent, 'end', text=child.function_name, values=(child.file_path, child.line_number))
            self.populate_tree(tree, tree_node, child)

    def on_tree_select(self, event):
        selected_item = self.tree.selection()[0]
        file_path = self.tree.item(selected_item, "values")[0]
        line_number = int(self.tree.item(selected_item, "values")[1])

        self.display_code(file_path, line_number)

    def display_code(self, file_path, line_number, context=10):
        """
        Load and display the specified line and a few lines of context from the file.
        :param file_path: Path to the code file.
        :param line_number: The line number to highlight.
        :param context: Number of lines before and after the target line to display.
        """
        try:
            with open(file_path, 'r') as file:
                lines = file.readlines()

            # Calculate the range of lines to display
            start_line = max(0, line_number - context - 1)
            end_line = min(len(lines), line_number + context)
            displayed_lines = lines[start_line:end_line]

            self.code_display.delete(1.0, tk.END)
            self.code_display.insert(tk.END, "".join(displayed_lines))

            # Adjust the highlight to match the displayed lines
            highlight_line = line_number - start_line
            self.code_display.see(f"{highlight_line}.0")
            self.code_display.tag_add("highlight", f"{highlight_line}.0", f"{highlight_line}.end")
            self.code_display.tag_configure("highlight", background="yellow")
        except FileNotFoundError:
            self.code_display.insert(tk.END, f"File not found: {file_path}")

def start_gui_app():
    log_file_path = "/root/vescale_prj/veScale/test/parallel/pipeline/instruction/logs-test_schedule-host_f78e8e970a17-pid_199624-py/tracing-test_schedule-20240829_071457.log"
    logs = parse_log_file(log_file_path)

    call_stack_tree = CallStackTree()
    for log in logs:
        function_name, file_path, line_number, operation = log
        if operation == "push":
            call_stack_tree.push(function_name, file_path, line_number)
        elif operation == "pop":
            call_stack_tree.pop()

    root = tk.Tk()
    app = CodeReviewApp(root, call_stack_tree)
    root.mainloop()

# Start the Tkinter app
display("Starting Tkinter app...")
start_gui_app()